In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
#!pip install dask[dataframe]
#!pip install pandas --upgrade
# We have to do this as dask is bad at guessing the types of objects for some reason
# Also, if you are having trouble with running the dataframe and continuously get 
# Fatal Python error: GC object already tracked, try to update pandas
data_types = {
    'ID':  np.int64,
    'Case Number':  object,
    'Date': object,
    'Block': object,
    'IUCR': object,
    'Primary Type': object,
    'Description':  object,
    'Location Description': object,
    'Arrest':  bool,
    'Domestic': bool,
    'Beat': np.int64,
    'District': np.float64,
    'Ward': np.float64,
    'Community Area': np.float64,
    'FBI Code': object,
    'X Coordinate': np.float64,
    'Y Coordinate': np.float64,
    'Year': np.int64,
    'Updated On': object,
    'Latitude': np.float64,
    'Longitude': np.float64,
    'Location': object
}
df = pd.read_csv('crimes.csv')

# get subset of crimes -- with over ~100 entries, the geolocator.geocode() call times out
#crimes_sub  = df[:50]
#crimes_sub['Arrest'] = crimes_sub['Arrest'].astype(int)
#crimes_sub['Domestic'] = crimes_sub['Domestic'].astype(int)
#crimes_sub.head()


In [25]:
df.sort_values(['Year'], ascending=True)
df.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
6040754,8677449,HV352762,06/23/2012 11:30:00 PM,003XX E 109TH ST,0820,THEFT,$500 AND UNDER,STREET,False,False,...,9,49,06,1180290,1832757,2012,02/04/2016 06:33:39 AM,41.696346,-87.615506,"(41.69634565, -87.615506296)"
6040755,8677450,HV352661,06/19/2012 09:00:00 AM,043XX W ROOSEVELT RD,1120,DECEPTIVE PRACTICE,FORGERY,RESIDENCE,False,False,...,24,29,10,1147655,1894351,2012,02/04/2016 06:33:39 AM,41.866054,-87.733423,"(41.866053861, -87.73342347)"
6040756,8677451,HV352714,06/25/2012 03:15:00 PM,057XX N RICHMOND ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,False,False,...,40,2,05,1155656,1937939,2012,02/04/2016 06:33:39 AM,41.985506,-87.702874,"(41.985505539, -87.7028745)"
6040757,8677452,HV352683,06/25/2012 05:45:00 PM,009XX W ARGYLE ST,0880,THEFT,PURSE-SNATCHING,SIDEWALK,True,False,...,48,3,06,1169095,1933605,2012,02/04/2016 06:33:39 AM,41.973331,-87.653573,"(41.973330841, -87.653573434)"
6040758,8677454,HV352397,06/25/2012 12:45:00 PM,058XX N LINCOLN AVE,0560,ASSAULT,SIMPLE,"SCHOOL, PUBLIC, GROUNDS",False,False,...,40,2,08A,1155809,1938617,2012,02/04/2016 06:33:39 AM,41.987363,-87.702293,"(41.987362912, -87.702293402)"


In [24]:
df.tail()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
6040754,8677449,HV352762,06/23/2012 11:30:00 PM,003XX E 109TH ST,0820,THEFT,$500 AND UNDER,STREET,False,False,...,9,49,06,1180290,1832757,2012,02/04/2016 06:33:39 AM,41.696346,-87.615506,"(41.69634565, -87.615506296)"
6040755,8677450,HV352661,06/19/2012 09:00:00 AM,043XX W ROOSEVELT RD,1120,DECEPTIVE PRACTICE,FORGERY,RESIDENCE,False,False,...,24,29,10,1147655,1894351,2012,02/04/2016 06:33:39 AM,41.866054,-87.733423,"(41.866053861, -87.73342347)"
6040756,8677451,HV352714,06/25/2012 03:15:00 PM,057XX N RICHMOND ST,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,False,False,...,40,2,05,1155656,1937939,2012,02/04/2016 06:33:39 AM,41.985506,-87.702874,"(41.985505539, -87.7028745)"
6040757,8677452,HV352683,06/25/2012 05:45:00 PM,009XX W ARGYLE ST,0880,THEFT,PURSE-SNATCHING,SIDEWALK,True,False,...,48,3,06,1169095,1933605,2012,02/04/2016 06:33:39 AM,41.973331,-87.653573,"(41.973330841, -87.653573434)"
6040758,8677454,HV352397,06/25/2012 12:45:00 PM,058XX N LINCOLN AVE,0560,ASSAULT,SIMPLE,"SCHOOL, PUBLIC, GROUNDS",False,False,...,40,2,08A,1155809,1938617,2012,02/04/2016 06:33:39 AM,41.987363,-87.702293,"(41.987362912, -87.702293402)"


In [3]:
# get data from 2008 to 2012
data08_12 = df[df.Year.isin(range(2008, 2012))]
data08_12.shape
#data08_12.to_csv("data08_12")

(1540890, 22)

In [27]:
data08_12.shape
#data08_12.to_csv("data08_12.csv")
data08_12.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
6039393,8675358,HV350718,05/28/2010 12:00:00 AM,035XX W MC LEAN AVE,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,...,26,22,20,1152355,1913392,2010,02/04/2016 06:33:39 AM,41.918213,-87.715666,"(41.918212853, -87.715666088)"
6039429,8675396,HV350670,12/12/2011 11:00:00 PM,058XX W WILSON AVE,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,True,...,38,15,02,1136396,1929985,2011,02/04/2016 06:33:39 AM,41.964046,-87.773903,"(41.964046235, -87.773903451)"
6039457,8675427,HV350793,11/01/2011 12:00:00 PM,007XX E 101ST ST,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,False,...,8,50,26,1183159,1838065,2011,02/04/2016 06:33:39 AM,41.710845,-87.604837,"(41.710845388, -87.604837475)"
6040347,8676822,HV352128,03/07/2011 12:01:00 AM,028XX W 25TH PL,0810,THEFT,OVER $500,APARTMENT,True,False,...,12,30,06,1157846,1886955,2011,02/04/2016 06:33:39 AM,41.845557,-87.696213,"(41.845556924, -87.696212817)"
6040556,8677214,HV352403,02/15/2011 08:00:00 AM,024XX W FARRAGUT AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,False,...,40,4,06,1159179,1934787,2011,02/04/2016 06:33:39 AM,41.976784,-87.690004,"(41.97678442, -87.690004192)"


In [30]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

# Don't need this
#
# Gets full address using block address with "Chicago" appended
# Extracts zip code from full address
#
# TODO: address hyphenated zip code 60609-2524 parsed as two zip codes
#
def getZip(row):
    location = geolocator.geocode(row['Block'] + " Chicago", timeout=None)
    if location is None: 
        return 0
    address = location.address.replace(',', "")
    return next(iter([int(s) for s in address.split() if s.isdigit()]), 0)

# Add zip code column with zip code of crime
#data08_12['Zip Code'] = data08_12.apply(getZip, axis=1, reduce=False)
#data08_12.shape

In [7]:
# Socioeconomic stats by community area 
# https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2
#
poverty = pd.read_csv('poverty.csv')
poverty.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [28]:
# 5-01-2016: using zip codes results in geocoding timeout

# Mapping of community area numbers to zip codes 
# https://docs.google.com/spreadsheets/d/1YzfGnfeiLdYtbN8IoOctPZPJRCAms_gW6g-OTvkGAZU/edit#gid=0
#

#zip_codes = pd.read_csv('zipcodes.csv')
#zip_codes.head()

In [29]:
# don't need this

# Merge socioeconomic stats with community area numbers
#poverty_by_ca = pd.merge(poverty, zip_codes, how='left', on='Community Area Number')

# get avg stats for zip code (each zip code includes multiple community areas)
# methodology not ideal -- obscures variance in community areas, takes mean of community area number (could fix by splitting table)
# --> granularity/scope of location data = challenge to talk about in presentation
#poverty_by_zip = poverty_by_ca.groupby('Zip Code').mean()
#poverty_by_zip.head()
#poverty_by_ca.head()

In [26]:
# Merge crimes with socioeconomic data for relevant community area
#crimes_with_poverty = pd.merge(data08_12, poverty, how='left', left_on='Community Area', right_on='Community Area Number')

crimes_with_poverty = pd.merge(data08_12, poverty, how='left', left_on='Community Area', right_on='Community Area Number')
crimes_with_poverty.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Location,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,8677678,HV351159,06/02/2010 09:00:00 AM,086XX W FOSTER AVE,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,...,"(41.973771478, -87.842226779)",76,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24
1,8678050,HV349526,06/27/2011 12:44:00 PM,002XX W MADISON ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,OTHER,False,False,...,"(41.881944424, -87.634195294)",32,Loop,1.5,14.7,5.7,3.1,13.5,65526,3
2,8678122,HV353372,12/11/2011 02:00:00 PM,052XX N MOODY AVE,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,"(41.976282262, -87.782332131)",11,Jefferson Park,2.7,8.6,12.4,13.4,35.5,27751,25
3,8678213,HV353504,08/12/2011 12:00:00 PM,112XX S WALLACE ST,0810,THEFT,OVER $500,"SCHOOL, PUBLIC, BUILDING",False,False,...,"(41.68995741, -87.637460623)",49,Roseland,2.5,19.8,20.3,16.9,41.2,17949,52
4,8678564,HV353442,12/07/2011 08:00:00 AM,039XX W 79TH ST,0890,THEFT,FROM BUILDING,"SCHOOL, PUBLIC, BUILDING",False,False,...,"(41.749460005, -87.72089612)",70,Ashburn,4.0,10.4,11.7,17.7,36.9,23482,37


In [22]:
# Get number of crimes in each zip code
crimes_by_ca = crimes_with_poverty.groupby('Community Area').ID.count()
crimes_by_ca

# TODO: plot as histogram

Community Area
0         24
1      69429
2      79992
3      42064
4      51480
5      36124
6      92868
7     120065
8     194384
9       3064
10     26412
11     11960
12     11384
13     20620
14     56156
15     80192
16     55203
17     20292
18      7734
19    123236
20     28287
21     59740
22    198612
23    294000
24    368072
25    601608
26     26472
27     56906
28    289905
29    170700
       ...  
48     17434
49    131634
50     17253
51     20806
52      7985
53     77292
54     11450
55      6754
56     48304
57      5875
58     30594
59     12930
60      9975
61     32630
62      6228
63     15590
64      6301
65     11816
66     42181
67     47097
68     42630
69     38480
70     14657
71     45560
72      5584
73     18433
74      3482
75     12893
76      8319
77     14787
Name: ID, dtype: int64

In [ ]:
%matplotlib inline

# Get number of arrests in each zip code
arrests_by_ca = crimes_with_poverty.groupby('Community Area').Arrest.sum()
arrests_by_zip.hist

# TODO: plot as histogram